In [51]:
# BigFrames can work with tables that are too large to fit in the notebook memory.
# Look at the first 20 rows.
import pandas as pd
import numpy as np
from google.cloud import bigquery
client = bigquery.Client(project='cap-project-ninja-7')
df_market = client.query("SELECT * FROM `cap-project-ninja-7.CAP_PROJECT.STOCK_DAILY`").to_dataframe()
print(df_market)

           ts_code  trade_date   close    open    high     low  pre_close  \
0     your_ts_code    20231026  100.00   99.00  101.00   98.00      99.50   
1     your_ts_code    20231026  100.00   99.00  101.00   98.00      99.50   
2     your_ts_code    20231026  100.00   99.00  101.00   98.00      99.50   
3     your_ts_code    20231026  100.00   99.00  101.00   98.00      99.50   
4     your_ts_code    20231026  100.00   99.00  101.00   98.00      99.50   
...            ...         ...     ...     ...     ...     ...        ...   
6473     688390.SH    20230825  134.39  133.69  136.88  132.20     134.39   
6474     688520.SH    20231108   52.23   52.16   53.10   51.62      52.23   
6475     688520.SH    20240411   38.88   38.90   39.58   38.52      38.88   
6476     688578.SH    20231127   36.53   36.37   36.90   36.18      36.53   
6477     688696.SH    20240529   91.20   91.70   93.27   90.17      91.20   

      change  pct_chg       vol       amount  
0        0.5      0.5  10000

In [73]:
df_market['trade_date'] = pd.to_datetime(df_market['trade_date'], format='%Y%m%d')
stock_list = df_market['ts_code'].unique()
df_market = df_market.sort_values(by='trade_date', ascending=False)
print(df_market)
def get_transation(stockcode):
    df_sma = df_market[df_market['ts_code']==stockcode]
    dates = pd.DatetimeIndex(df_market['trade_date']).date.astype(np.datetime64)
    opens = df_market.open.values
    highs = df_market.high.values
    lows = df_market.low.values
    closes = df_market.close.values
    volumes = df_market.vol.values
    return dates,opens, highs, lows, closes, volumes

def calc_sma(N, dates, close_p):
    weights = np.ones(N) / N
    sma = np.convolve(close_p, weights, 'valid')
    return sma


code_list = []
trend_list = []
position_list = []
for stockcode in stock_list:
  if stockcode in ("000001.SH", "399300.SZ", "your_ts_code"):
    continue
  dates, opens, highs, lows, closes, volumes = get_transation(stockcode)
  N5 = 5
  sma_5 = calc_sma(N5, dates, closes)
  N13 = 13
  sma_13 = calc_sma(N13, dates, closes)
  N55 = 55
  sma_55 = calc_sma(N55, dates, closes)
  N100 = 100
  sma_100 = calc_sma(N100, dates, closes)
  N200 = 200
  sma_200 = calc_sma(N200, dates, closes)
  price_sort = np.array([sma_5[0], sma_13[0], sma_55[0], sma_100[0], sma_200[0]])
  if price_sort.argsort().tolist() == [4, 3, 2, 1, 0] and round((sma_5[0]/sma_200[0])-1, 4) < 0.35 and round((closes[0]/sma_200[0])-1, 4) > 0:
      trend = "perfect"
  elif price_sort.argsort().tolist() == [4, 3, 2, 1, 0] and round((sma_5[0]/sma_200[0])-1, 4) > 0.40 and round((closes[0]/sma_200[0])-1, 4) > 0:
      trend = "radical"
  elif (price_sort.argsort().tolist() == [3, 4, 2, 1, 0] or price_sort.argsort().tolist() == [4, 3, 2, 0, 1]) and round((sma_5[0]/sma_200[0])-1, 4) < 0.35 and round((closes[0]/sma_200[0])-1, 4) > 0:
      trend = "good"
  elif price_sort.argsort().tolist() == [0, 1, 2, 3, 4]:
      trend = "bad"
  else:
      trend = "flat"

  if np.max(highs[0:200]) == np.max(highs):
      if closes[0]/np.max(highs[0:200]) < 0.75:
          position = "perfect"
      else:
          position = "radical"
  else:
      if closes[0]/np.max(highs) < 0.75:
          position = "good"
      else:
          position = "bad"
  code_list.append(stockcode)
  trend_list.append(trend)
  position_list.append(position)

df_sma = pd.DataFrame({'ts_code':code_list, 'trend':trend_list, 'position':position_list})
print(df_sma)




        ts_code trade_date   close    open    high     low  pre_close  change  \
3061  300695.SZ 2024-08-15   36.51   35.61   36.65   35.40      35.83    0.68   
3517  300859.SZ 2024-08-15   26.08   25.82   26.45   25.75      25.96    0.12   
1611  002180.SZ 2024-08-15   23.68   23.89   24.00   23.35      23.80   -0.12   
2269  002865.SZ 2024-08-15   41.40   41.50   42.34   39.77      41.24    0.16   
586   000915.SZ 2024-08-15   27.27   27.16   27.60   27.07      27.31   -0.04   
...         ...        ...     ...     ...     ...     ...        ...     ...   
6069  688578.SH 2023-08-21   22.97   22.90   23.35   22.84      22.89    0.08   
2437  003031.SZ 2023-08-21  101.37  102.38  103.89  101.00     102.38   -1.01   
1066  001269.SZ 2023-08-21   52.12   51.88   52.70   51.88      52.04    0.08   
3050  300695.SZ 2023-08-21   46.90   46.58   47.99   46.58      47.14   -0.24   
1898  002432.SZ 2023-08-21   34.32   34.71   35.09   34.08      34.60   -0.28   

      pct_chg        vol   

In [64]:
from google.cloud import storage

bucket_name = 'cap_project'
file_path = 'SMA/SMA.csv'
local_file_path = 'SMA.csv'

csv_file = df_sma.to_csv(index=False)

storage_client = storage.Client(project='cap-project-ninja-7')
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_path)
blob.upload_from_string(csv_file)

# blob.upload_from_filename(local_file_path)



In [75]:
import math
stock_list = df_market['ts_code'].unique()

df_sharp = df_market[df_market['trade_date'] >= '2024-08-01']

def sharp_index(df):
  df['ex_pct_close'] = df['pct_chg'] - 0.022/252
  sharp = (df['ex_pct_close'].mean() * math.sqrt(252)) / \
            df['ex_pct_close'].std()
  return sharp

code_list = []
sharp_list = []
for stockcode in stock_list:
  if stockcode in ("000001.SH", "399300.SZ", "your_ts_code"):
    continue
  print(stockcode)
  sharp = sharp_index(df_sharp[df_sharp['ts_code'] == stockcode])
  print(sharp)
  code_list.append(stockcode)
  sharp_list.append(sharp)

df_sharp_index = pd.DataFrame({'ts_code':code_list, 'sharp':sharp_list})
print(df_sharp_index)


300695.SZ
1.8440975832930342
300859.SZ
-1.557141036464987
002180.SZ
-14.990597230503298
002865.SZ
3.069865788889769
000915.SZ
-4.210382070483904
300827.SZ
4.79622156096209
603688.SH
-5.780998047147048
002466.SZ
-6.816795642340006
688696.SH
-9.153275481172656
001269.SZ
1.7895299146706758
300274.SZ
0.030241339997263417
002432.SZ
0.9640176074716065
000921.SZ
-12.77794074079875
000651.SZ
2.3532608348660964
603031.SH
-7.9825918845446235
001283.SZ
-8.706871830943452
688045.SH
-3.9224817233287537
603199.SH
1.633582469440324
688036.SH
-5.46564735400771
301096.SZ
-3.5757671926734975
688520.SH
-4.812978985221183
688408.SH
-9.273594365756185
688390.SH
-2.1826193247535595
688578.SH
-4.742908905647297
003031.SZ
-9.722666061328196
      ts_code      sharp
0   300695.SZ   1.844098
1   300859.SZ  -1.557141
2   002180.SZ -14.990597
3   002865.SZ   3.069866
4   000915.SZ  -4.210382
5   300827.SZ   4.796222
6   603688.SH  -5.780998
7   002466.SZ  -6.816796
8   688696.SH  -9.153275
9   001269.SZ   1.78953

<ipython-input-75-35e8528ee8e4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ex_pct_close'] = df['pct_chg'] - 0.022/252
<ipython-input-75-35e8528ee8e4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ex_pct_close'] = df['pct_chg'] - 0.022/252
<ipython-input-75-35e8528ee8e4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [76]:
from google.cloud import storage

bucket_name = 'cap_project'
file_path = 'SHARP/SHARP.csv'
local_file_path = 'SHARP.csv'

csv_file = df_sharp_index.to_csv(index=False)

storage_client = storage.Client(project='cap-project-ninja-7')
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_path)
blob.upload_from_string(csv_file)